In [18]:
##import library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [19]:
###set parameters and other variables to train the model.
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = False
seed = 1
log_interval = 200

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print("set vars and device done")

set vars and device done


In [20]:
#Prepare Data Loader for Training and Validation

transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize((0.1307,), (0.3081,))])

train_loader = torch.utils.data.DataLoader(
  datasets.MNIST('../data', train=True, download=True, 
                 transform=transform), 
    batch_size = batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True,
                 transform=transform), 
    batch_size=test_batch_size, shuffle=False, **kwargs)

In [21]:
###### define CNN model!!!

class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv1 = nn.Conv2d(1,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.conv3 = nn.Conv2d(64,128,3,1)
        
        self.dropout1 = nn.Dropout2d(0.5)
        self.dropout2 = nn.Dropout2d(0.5)
        self.dropout3 = nn.Dropout2d(0.5)

        self.fc = nn.Linear(11*11*128,10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.dropout1(x)
        x = F.relu(self.conv2(x))
        x = self.dropout2(x)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,2)
        x = self.dropout3(x)

        x = torch.flatten(x,1)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1) 
        return output

In [22]:
model = CNN2().to(device)


In [23]:
#Define Train function and Test function to validate.

def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [24]:
# Train and Test the model and save it.
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, 11):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)
torch.save(model, './model_SGD.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.291814
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.790128
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.410506
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.377077
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.184529

Test set: Average loss: 0.1346, Accuracy: 9604/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.233337
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.174443
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.209403
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.141317
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.050942

Test set: Average loss: 0.0846, Accuracy: 9727/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.182065
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.135111
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.204360
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.160706
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.127712

Test set: Average loss: 0.0656, Accuracy: 9808/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.057535
Train Epoch: 4 [12800/60000 (21%)]	Lo

In [25]:
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, 11):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)
torch.save(model, './model_Adam.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.075189
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.252024
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.356209
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.415694
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.381688

Test set: Average loss: 0.1131, Accuracy: 9640/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.411489
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.200525
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.228363
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.425515
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.060003

Test set: Average loss: 0.0944, Accuracy: 9700/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.048030
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.055859
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.106721
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.074135
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.114638

Test set: Average loss: 0.0889, Accuracy: 9724/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.181531
Train Epoch: 4 [12800/60000 (21%)]	Lo

In [26]:
optimizer = optim.Adadelta(model.parameters(), lr=lr)
for epoch in range(1, 11):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)
torch.save(model, './model_Adadelta.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.294406
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.447166
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.138070
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.236507
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.085040

Test set: Average loss: 0.0923, Accuracy: 9730/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.259852
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.162096
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.059797
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.420864
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.322792

Test set: Average loss: 0.0918, Accuracy: 9733/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.186382
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.140912
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.093022
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.152915
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.170268

Test set: Average loss: 0.0913, Accuracy: 9734/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.116082
Train Epoch: 4 [12800/60000 (21%)]	Lo